Much of the code for this notebook is sourced from the original underlying base model scCraft. This ensures reproducibilty of the original results in comparison to the new approach. The github link is available here: https://github.com/ch2343/scCRAFT/tree/main, and a link to the publication is available here: https://www.nature.com/articles/s42003-025-07988-y. 

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from scib.utils import *
import torch

from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
import torch.nn as nn
import torch.nn.functional as F  # noqa: N812
import torch.optim as optim

from typing import Union

import jax.numpy as jnp
from torch.autograd import grad
from torch.distributions import Normal
from torch.distributions import kl_divergence as kl

from itertools import combinations
import random

from torch.utils.data import DataLoader, TensorDataset
import scipy
import scib
import time

torch.backends.cudnn.benchmark = True

In [ ]:
# Utility functions

def weights_init_normal(m):
    classname = m.__class__.__name__
    # Skip if it's an instance of _DomainSpecificBatchNorm
    if classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


def generate_balanced_dataloader(adata, batch_size, batch_key="batch"):
    if not adata.obs_names.is_unique:
        print("Error: Indices are not unique!")
        raise AssertionError(
            "Indices are not unique. Please ensure the indices are unique before proceeding."
        )
    # Map unique batch keys to integers
    unique_batches = adata.obs[batch_key].unique()
    batch_to_int = {batch: i for i, batch in enumerate(unique_batches)}
    unsupervised_labels1 = adata.obs["leiden1"].cat.codes.values
    unsupervised_labels2 = adata.obs["leiden2"].cat.codes.values
    # Separate the dataset by batches and sample indices
    batch_indices = []
    batch_labels_list = []
    for batch in unique_batches:
        # Find the indices for the current batch
        batch_indices_in_adata = adata.obs[adata.obs[batch_key] == batch].index

        # Sample indices from the current batch
        if len(batch_indices_in_adata) >= batch_size:
            sampled_indices = np.random.choice(batch_indices_in_adata, batch_size, replace=False)
        else:
            # If not enough cells, sample with replacement
            sampled_indices = np.random.choice(batch_indices_in_adata, batch_size, replace=True)

        # Get the integer positions of the sampled indices
        sampled_indices_pos = [adata.obs_names.get_loc(idx) for idx in sampled_indices]
        batch_indices.extend(sampled_indices_pos)

        # Map the batch keys to integers and add to the label list
        batch_labels_list.extend([batch_to_int[batch]] * batch_size)

    # Extract the feature data
    x_sampled = adata.X[batch_indices, :]

    # Convert features to tensor
    if isinstance(x_sampled, np.ndarray):
        x_tensor = torch.tensor(x_sampled, dtype=torch.float32)
    else:  # if it's a sparse matrix
        x_tensor = torch.tensor(x_tensor.toarray(), dtype=torch.float32)

    # Convert batch labels to tensor
    v_tensor = torch.tensor(batch_labels_list, dtype=torch.int64)
    label_tensor1 = torch.tensor(unsupervised_labels1[batch_indices], dtype=torch.int64)
    label_tensor2 = torch.tensor(unsupervised_labels2[batch_indices], dtype=torch.int64)

    # Create a TensorDataset and DataLoader
    combined_dataset = TensorDataset(x_tensor, v_tensor, label_tensor1, label_tensor2)
    dataloader = DataLoader(combined_dataset, batch_size=batch_size * 2, shuffle=True)

    return dataloader


# Example usage (assuming `adata` is your AnnData object):
# data_loader = generate_balanced_dataloader(adata, batch_size=256, batch_key='batch')


def count_labels_per_batch(labels, batch_ids):
    unique_batches = batch_ids.unique()
    label_counts_per_batch = {
        batch: (
            labels[batch_ids == batch].unique(),
            torch.stack(
                [
                    (labels[batch_ids == batch] == label).sum()
                    for label in labels[batch_ids == batch].unique()
                ]
            ),
        )
        for batch in unique_batches
    }
    return label_counts_per_batch


def create_triplets(
    embeddings, labels, labels_high, batch_ids, margin=1.0, num_triplets_per_label=15
):
    label_counts_per_batch = count_labels_per_batch(labels, batch_ids)
    triplets = []

    for batch_id, (unique_labels, _) in label_counts_per_batch.items():
        for label in unique_labels:
            indices_with_label = (labels == label) & (batch_ids == batch_id)
            indices_without_label = (labels != label) & (batch_ids == batch_id)

            positive_pairs = list(combinations(torch.where(indices_with_label)[0], 2))
            negative_indices = torch.where(indices_without_label)[0]

            # Randomly sample triplets
            sampled_positive_pairs = random.sample(
                positive_pairs, min(num_triplets_per_label, len(positive_pairs))
            )
            # Ensure the number of negative samples doesn't exceed the available negatives
            num_negative_samples = min(len(sampled_positive_pairs), len(negative_indices))
            sampled_negative_indices = random.sample(list(negative_indices), num_negative_samples)

            for (anchor_idx, positive_idx), negative_idx in zip(
                sampled_positive_pairs, sampled_negative_indices
            ):
                anchor, positive, negative = (
                    embeddings[anchor_idx],
                    embeddings[positive_idx],
                    embeddings[negative_idx],
                )
                if labels_high[anchor_idx] != labels_high[positive_idx]:
                    # Only consider anchor-negative pair for loss calculation
                    # triplet_loss = torch.relu(-torch.norm(anchor - negative) + margin)
                    continue
                else:
                    # Standard triplet loss calculation for anchor-positive and anchor-negative
                    triplet_loss = torch.relu(
                        torch.norm(anchor - positive) - torch.norm(anchor - negative) + margin
                    )
                    triplets.append(triplet_loss)

    if triplets:
        return torch.mean(torch.stack(triplets))
    else:
        return torch.tensor(0.0)


def set_seed(seed):
    """
    Set the seed for different packages to ensure reproducibility.

    Parameters:
    seed (int): The seed number.
    """
    # Set seed for NumPy
    np.random.seed(seed)

    # Set seed for Python's built-in random module
    random.seed(seed)

    # Set seed for PyTorch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU setups


def multi_resolution_cluster(adata, resolution1=0.5, resolution2=7, method="Louvain"):
    """
    Performs PCA, neighbors calculation, and clustering with specified resolutions and method.

    Parameters:
    - adata: AnnData object containing single-cell data.
    - resolution1: float, the resolution parameter for the first clustering.
    - resolution2: float, the resolution parameter for the second clustering.
    - method: str, clustering method to use ("Louvain" or "Leiden").

    The function updates `adata` in place, adding two new columns to `adata.obs`:
    - 'leiden1': contains cluster labels from the first clustering.
    - 'leiden2': contains cluster labels from the second clustering.
    """
    # Perform PCA
    sc.tl.pca(adata, n_comps=50)
    # Compute neighbors using the PCA representation
    sc.pp.neighbors(adata, use_rep="X_pca")

    # Determine the clustering function based on the method
    if method.lower() == "louvain":
        clustering_function = sc.tl.louvain
    elif method.lower() == "leiden":
        clustering_function = sc.tl.leiden
    else:
        raise ValueError("Method should be 'Louvain' or 'Leiden'")

    # Perform the first round of clustering
    clustering_function(adata, resolution=resolution1)
    adata.obs["leiden1"] = adata.obs[method.lower()]

    # Perform the second round of clustering with a different resolution
    clustering_function(adata, resolution=resolution2)
    adata.obs["leiden2"] = adata.obs[method.lower()]


# Loader only for Visualization
def generate_adata_to_dataloader(adata, batch_size=2048):
    x_dense = adata.X.toarray() if isinstance(adata.X, scipy.sparse.spmatrix) else adata.X

    x_tensor = torch.tensor(x_dense, dtype=torch.float32)

    # Create a DataLoader for batch-wise processing
    dataset = torch.utils.data.TensorDataset(
        x_tensor, torch.arange(len(x_tensor))
    )  # include indices
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    return data_loader


In [ ]:
# Net + Loss function

def log_nb_positive(
    x: Union[torch.Tensor, jnp.ndarray],
    mu: Union[torch.Tensor, jnp.ndarray],
    theta: Union[torch.Tensor, jnp.ndarray],
    eps: float = 1e-8,
    log_fn: callable = torch.log,
    lgamma_fn: callable = torch.lgamma,
):
    """Log likelihood (scalar) of a minibatch according to a nb model.

    Parameters
    ----------
    x
        data
    mu
        mean of the negative binomial (has to be positive support) (shape: minibatch x vars)
    theta
        inverse dispersion parameter (has to be positive support) (shape: minibatch x vars)
    eps
        numerical stability constant
    log_fn
        log function
        lgamma_fn
        log gamma function
    """
    log = log_fn
    lgamma = lgamma_fn
    log_theta_mu_eps = log(theta + mu + eps)
    res = (
        theta * (log(theta + eps) - log_theta_mu_eps)
        + x * (log(mu + eps) - log_theta_mu_eps)
        + lgamma(x + theta)
        - lgamma(theta)
        - lgamma(x + 1)
    )

    return res


def reparameterize_gaussian(mu, var):
    return Normal(mu, var.sqrt()).rsample()


class Encoder(nn.Module):
    def __init__(self, p_dim, latent_dim):
        super().__init__()
        # Define the architecture
        self.fc1 = nn.Linear(p_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc_mean = nn.Linear(512, latent_dim)  # Output layer for mean
        self.fc_var = nn.Linear(512, latent_dim)  # Output layer for variance
        # self.fc_library = nn.Linear(512, 1)        # Output layer for library size
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)

    def forward(self, x, warmup):
        # Forward pass through the network
        x = self.fc1(x)
        x = self.relu(self.bn1(x))

        x = self.fc2(x)
        x = self.relu(self.bn2(x))

        # Separate paths for mean, variance, and library size
        q_m = self.fc_mean(x)
        q_v = torch.exp(self.fc_var(x)) + 1e-4
        # library = self.fc_library(x)  # Predicted log library size

        z = reparameterize_gaussian(q_m, q_v)

        return q_m, q_v, z


class Decoder(nn.Module):
    def __init__(self, p_dim, v_dim, latent_dim=256):
        super().__init__()
        self.relu = nn.ReLU()

        # Main decoder pathway
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + v_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, p_dim),
        )

        # Additional pathway for the batch effect (ec)
        self.decoder_ec = nn.Sequential(
            nn.Linear(v_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, p_dim),
        )

        # Parameters for ZINB distribution
        self.px_scale_decoder = nn.Linear(p_dim, p_dim)  # mean (rate) of ZINB
        self.px_r_decoder = nn.Linear(p_dim, p_dim)  # dispersion

    def forward(self, z, ec):
        # Main decoding
        z_ec = torch.cat((z, ec), dim=-1)
        decoded = self.decoder(z_ec)
        decoded_ec = self.decoder_ec(ec)

        # Combining outputs
        combined = self.relu(decoded + decoded_ec)

        # NB parameters with safe exponential

        px_scale = torch.exp(self.px_scale_decoder(combined))
        px_r = torch.exp(self.px_r_decoder(combined))

        # Scale the mean (px_scale) with the predicted library size
        px_rate = px_scale

        return px_rate, px_r


class VAE(nn.Module):
    def __init__(self, p_dim, v_dim, latent_dim):
        super().__init__()
        self.encoder = Encoder(p_dim, latent_dim)
        self.decoder = Decoder(p_dim, v_dim, latent_dim)

    def forward(self, x, ec, warmup):
        # Encoding
        q_m, q_v, z = self.encoder(x, warmup)

        # Decoding
        px_scale, px_r = self.decoder(z, ec)

        # Reconstruction Loss
        # reconst_loss = F.mse_loss(px_scale, x)
        reconst_loss = -log_nb_positive(x, px_scale, px_r)
        # KL Divergence
        mean = torch.zeros_like(q_m)
        scale = torch.ones_like(q_v)
        kl_divergence = kl(Normal(q_m, torch.sqrt(q_v)), Normal(mean, scale)).sum(dim=1)

        return reconst_loss, kl_divergence, z, px_scale


class CrossEntropy(nn.Module):
    def __init__(self, reduction="mean"):
        super().__init__()
        self.reduction = reduction

    def forward(self, output, target):
        # Apply log softmax to the output
        log_preds = F.log_softmax(output, dim=-1)

        # Compute the negative log likelihood loss
        loss = F.nll_loss(log_preds, target, reduction=self.reduction)

        return loss


class WassersteinLoss(nn.Module):
    def __init__(self, reduction="mean"):
        super().__init__()
        self.reduction = reduction

    def forward(self, output, batch_ids):
        source = output[batch_ids != 0]
        target = output[batch_ids == 0]

        # Compute the Wasserstein loss
        loss = -1 * target.mean() + source.mean()

        return loss


class MultiClassWassersteinLoss(nn.Module):
    def __init__(self, reduction="mean"):
        super().__init__()
        self.reduction = reduction

    def forward(self, output, batch_ids):
        """
        Args:
            output: Tensor of shape [B, K] - critic scores for each class.
            batch_ids: Tensor of shape [B] - true domain IDs (0 to K-1).
        Returns:
            Wasserstein loss encouraging domain confusion.
        """
        num_domains = output.shape[1]  # number of classes/domains
        loss = 0.0
        total = 0

        for k in range(num_domains):
            mask_k = batch_ids == k
            if mask_k.sum() == 0:
                continue

            # Scores for domain k samples from the k-th output head
            d_kk = output[mask_k, k]  # true class head for samples from domain k

            # Scores for domain k samples from all other heads
            d_kj = output[mask_k]  # [n_k, K]
            mask_other = torch.ones(num_domains, dtype=torch.bool, device=output.device)
            mask_other[k] = False
            d_kj_others = d_kj[:, mask_other]  # [n_k, K-1]

            # Wasserstein-style loss: true score - mean of other scores
            diff = d_kk.mean() - d_kj_others.mean()
            loss += diff
            total += 1

        loss = loss / total

        return loss


def gradient_penalty(discriminator, real_samples, fake_samples, device="cpu"):
    """Computes gradient penalty for WGAN-GP"""
    batch_size = real_samples.size(0)
    epsilon = torch.rand(batch_size, 1, device=device)
    epsilon = epsilon.expand_as(real_samples)

    # if there is a mismatch in shape, subsample the larger tensor
    if fake_samples.shape != real_samples.shape:
        if fake_samples.shape[0] > real_samples.shape[0]:
            perm = torch.randperm(fake_samples.shape[0], device=device)
            fake_samples = fake_samples[perm[: real_samples.shape[0]]]
        elif real_samples.shape[0] > fake_samples.shape[0]:
            perm = torch.randperm(real_samples.shape[0], device=device)
            real_samples = real_samples[perm[: fake_samples.shape[0]]]

    # Interpolate between real and fake samples
    interpolated = epsilon * real_samples + (1 - epsilon) * fake_samples
    interpolated.requires_grad_(True)

    # Forward pass
    d_interpolated = discriminator(interpolated, None, generator=True)

    # Forcing scalar output if necessary
    if d_interpolated.dim() > 1:
        d_interpolated = d_interpolated.view(-1)

    # Compute gradients w.r.t. interpolated
    gradients = grad(
        outputs=d_interpolated,
        inputs=interpolated,
        grad_outputs=torch.ones_like(d_interpolated),
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]

    # Compute the gradient norm
    gradients = gradients.view(batch_size, -1)
    grad_norm = gradients.norm(2, dim=1)

    # Compute the penalty
    penalty = ((grad_norm - 1) ** 2).mean()
    return penalty


def multi_class_gradient_penalty(critic, z, batch_ids, lambda_gp=10.0):
    """
    Computes the multi-class gradient penalty for a multi-output critic.

    Args:
        critic: A callable that maps latent vectors z to shape [B, K] (K = num domains).
        z: Latent vectors, shape [B, latent_dim].
        batch_ids: Tensor of shape [B] with domain labels (0 to K-1).
        lambda_gp: Weight of gradient penalty.

    Returns:
        Scalar gradient penalty loss.
    """
    b, latent_dim = z.shape
    critic_out = critic(z, batch_ids=None).shape[1]
    gp_total = 0.0
    device = z.device
    total_classes = 0

    for k in range(critic_out):
        # Get samples from domain k
        mask_k = batch_ids == k
        if mask_k.sum() == 0:
            continue

        z_k = z[mask_k]
        z_ref = z[torch.randperm(z.size(0))[: z_k.size(0)]]

        # Interpolate
        epsilon = torch.rand(z_k.size(0), 1, device=device)
        epsilon = epsilon.expand_as(z_k)
        z_hat = epsilon * z_k + (1 - epsilon) * z_ref
        z_hat.requires_grad_(True)

        # Forward pass through critic
        out = critic(z_hat, batch_ids=None)  # [B_k, K]
        out_k = out[:, k].sum()  # Only the k-th head

        # Compute gradients
        grad = torch.autograd.grad(
            outputs=out_k, inputs=z_hat, create_graph=True, retain_graph=True, only_inputs=True
        )[0]

        # Compute L2 norm of gradients
        grad_norm = grad.view(grad.size(0), -1).norm(2, dim=1)
        gp = ((grad_norm - 1) ** 2).mean()

        gp_total += gp
        total_classes += 1

    if total_classes == 0:
        return torch.tensor(0.0, device=z.device)

    return lambda_gp * gp_total / total_classes


class Discriminator(nn.Module):
    def __init__(self, n_input, domain_number, critic=False):
        super().__init__()
        n_hidden = 128
        self.critic = critic
        # Define layers
        self.fc1 = nn.Linear(n_input, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, domain_number)

        if self.critic:
            # If using critic, use Wasserstein loss
            self.loss = MultiClassWassersteinLoss()
        else:
            # If not using critic, use cross-entropy loss
            self.loss = CrossEntropy()

    def forward(self, x, batch_ids, generator=False):
        # Forward pass through layers
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        output = self.fc3(h)

        if batch_ids is None:
            # If batch_ids is None, return the output directly
            return output

        discriminator_loss = self.loss(output, batch_ids)

        gp_loss = 0.0

        if self.loss.reduction == "mean":
            discriminator_loss = discriminator_loss.mean()
        elif self.loss.reduction == "sum":
            discriminator_loss = discriminator_loss.sum()
        if self.critic:
            gp_loss = multi_class_gradient_penalty(self, x, batch_ids)

        return discriminator_loss, gp_loss


In [ ]:
# Main training class
class SCIntegrationModel(nn.Module):
    def __init__(self, adata, batch_key, z_dim, critic, seed):
        super().__init__()
        self.p_dim = adata.shape[1]
        self.z_dim = z_dim
        self.v_dim = np.unique(adata.obs[batch_key]).shape[0]

        # Correctly initialize VAE with p_dim, v_dim, and latent_dim
        self.VAE = VAE(p_dim=self.p_dim, v_dim=self.v_dim, latent_dim=self.z_dim)
        self.D_Z = Discriminator(n_input=self.z_dim, domain_number=self.v_dim, critic=critic)
        self.mse_loss = torch.nn.MSELoss()

        # self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.device = "cpu"
        # Move models to CUDA if available
        self.VAE.to(self.device)
        self.D_Z.to(self.device)

        # Initialize weights
        if seed is not None:
            set_seed(seed)
        self.VAE.apply(weights_init_normal)
        self.D_Z.apply(weights_init_normal)

    def train_model(
        self,
        adata,
        batch_key,
        epochs,
        d_coef,
        kl_coef,
        triplet_coef,
        cos_coef,
        warmup_epoch,
        disc_iter,
    ):
        # Optimizer for VAE (Encoder + Decoder)
        optimizer_g = optim.Adam(self.VAE.parameters(), lr=0.001, weight_decay=0.0)
        # Optimizer for Discriminator
        optimizer_d_z = optim.Adam(self.D_Z.parameters(), lr=0.001, weight_decay=0.0)

        for epoch in range(epochs):
            data_loader = generate_balanced_dataloader(adata, batch_size=512, batch_key=batch_key)
            self.VAE.train()
            self.D_Z.train()
            all_losses = 0
            d_loss = 0
            t_loss = 0
            v_loss = 0
            warmup = epoch < warmup_epoch
            for _, (x, v, labels_low, labels_high) in enumerate(data_loader):
                x = x.to(self.device)
                v = v.to(self.device)
                labels_low = labels_low.to(self.device)
                labels_high = labels_high.to(self.device)
                batch_size = x.size(0)
                v_true = v
                v_one_hot = torch.zeros(batch_size, self.v_dim).to(x.device)
                # Use scatter_ to put 1s in the indices indicated by v
                v = v.unsqueeze(1)  # Ensure v is of shape [batch_size, 1] if it's not already

                v_one_hot.scatter_(1, v, 1).to(v.device)

                reconst_loss, kl_divergence, z, x_tilde = self.VAE(x, v_one_hot, warmup)
                reconst_loss = torch.clamp(reconst_loss, max=1e5)

                loss_cos = (
                    1 - torch.sum(F.normalize(x_tilde, p=2) * F.normalize(x, p=2), 1)
                ).mean()
                loss_vae = torch.mean(reconst_loss.mean() + kl_coef * kl_divergence.mean())

                for _ in range(disc_iter):
                    optimizer_d_z.zero_grad()
                    if self.D_Z.critic:
                        loss_d_z, gp = self.D_Z(z.detach(), v_true)
                    else:
                        loss_d_z, gp = self.D_Z(z.detach(), v_true)

                    loss_d_z += gp

                    loss_d_z.backward(retain_graph=True)
                    optimizer_d_z.step()

                optimizer_g.zero_grad()
                if self.D_Z.critic:
                    loss_da, gp = self.D_Z(z, v_true)
                else:
                    loss_da, gp = self.D_Z(z, v_true)

                triplet_loss = create_triplets(z, labels_low, labels_high, v_true, margin=5)

                if warmup:
                    all_loss = (
                        -0 * loss_da
                        + 1 * loss_vae
                        + gp
                        + triplet_coef * triplet_loss
                        + cos_coef * loss_cos
                    )
                else:
                    all_loss = (
                        -d_coef * loss_da
                        + 1 * loss_vae
                        + gp
                        + triplet_coef * triplet_loss
                        + cos_coef * loss_cos
                    )

                all_loss.backward()
                optimizer_g.step()
                all_losses += all_loss
                d_loss += loss_da
                t_loss += triplet_loss
                v_loss += loss_vae


def train_integration_model(
    adata,
    disc_iter,
    batch_key="batch",
    z_dim=256,
    epochs=150,
    d_coef=0.2,
    kl_coef=0.005,
    triplet_coef=1,
    cos_coef=20,
    warmup_epoch=50,
    critic=False,
    scale=None,
):
    number_of_cells = adata.n_obs
    number_of_batches = np.unique(adata.obs[batch_key]).shape[0]

    # Default number of epochs
    if epochs == 150:
        # Check if the number of cells goes above 100000
        if number_of_cells > 100000:
            calculated_epochs = int(1.5 * number_of_cells / (number_of_batches * 512))
            # If the calculated value is larger than the default, use it instead
            if calculated_epochs > epochs:
                epochs = calculated_epochs
    else:
        epochs = epochs
    model = SCIntegrationModel(
        adata=adata, batch_key=batch_key, z_dim=z_dim, critic=critic, seed=42)
    print(epochs)
    start_time = time.time()
    model.train_model(
        adata,
        batch_key=batch_key,
        epochs=epochs,
        d_coef=d_coef,
        kl_coef=kl_coef,
        triplet_coef=triplet_coef,
        cos_coef=cos_coef,
        warmup_epoch=warmup_epoch,
        disc_iter=disc_iter,
    )
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training completed in {training_time:.2f} seconds")

    model.VAE.eval()
    return model.VAE


def obtain_embeddings(adata, vae, dim=50, pca=True, seed=None):
    if seed is not None:
        set_seed(seed)

    vae.eval()
    data_loader = generate_adata_to_dataloader(adata)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_z = []
    all_indices = []

    for _, (x, indices) in enumerate(data_loader):
        x = x.to(device)
        _, _, z = vae.encoder(x, warmup=False)
        all_z.append(z)
        all_indices.extend(indices.tolist())

    all_z_combined = torch.cat(all_z, dim=0)
    all_indices_tensor = torch.tensor(all_indices)
    all_z_reordered = all_z_combined[all_indices_tensor.argsort()]
    all_z_np = all_z_reordered.cpu().detach().numpy()

    # Create anndata object with reordered embeddings
    adata.obsm["X_scCRAFT"] = all_z_np

    if pca:
        pca_model = PCA(n_components=dim)
        # Fit and transform the data
        x_sccraft_pca = pca_model.fit_transform(adata.obsm["X_scCRAFT"])
        # Store the PCA-reduced data back into adata.obsm
        adata.obsm["X_scCRAFT"] = x_sccraft_pca

    return adata


In [ ]:
def plot_umap_by_technology(adata, batch_key='tech', color_key='celltype', ncols=3, figsize_per_panel=(5, 5)):
    """
    Plot UMAP with consistent x and y scales and consistent colors for each technology/batch.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object with UMAP coordinates in obsm['X_umap']
    batch_key : str, default 'tech'
        Key in adata.obs containing batch/technology information
    color_key : str, default 'celltype'
        Key in adata.obs for coloring points
    ncols : int, default 3
        Maximum number of columns in subplot grid
    figsize_per_panel : tuple, default (5, 5)
        Size of each subplot panel
    
    Returns:
    --------
    None (displays plots)
    """

    
    sc.tl.umap(adata, min_dist=0.5)
    
    # Ensure cell types are categorical
    if not pd.api.types.is_categorical_dtype(adata.obs[color_key]):
        adata.obs[color_key] = adata.obs[color_key].astype('category')
    
    # Get unique technologies/batches and cell types
    technologies = adata.obs[batch_key].unique()
    cell_types = adata.obs[color_key].cat.categories
    
    # Create a consistent colormap for cell types
    cmap = plt.cm.get_cmap('tab20', len(cell_types))
    colors = [cmap(i) for i in range(len(cell_types))]
    color_dict = dict(zip(cell_types, colors))
    
    # Get the overall x and y limits from the full UMAP
    x_coords = adata.obsm['X_umap'][:, 0]
    y_coords = adata.obsm['X_umap'][:, 1]
    x_min, x_max = x_coords.min() - 0.5, x_coords.max() + 0.5
    y_min, y_max = y_coords.min() - 0.5, y_coords.max() + 0.5
    
    # Create subplots - adjust the number of columns based on preference
    n_techs = len(technologies)
    ncols = min(ncols, n_techs)
    nrows = (n_techs + ncols - 1) // ncols
    
    fig, axes = plt.subplots(nrows, ncols, figsize=(figsize_per_panel[0]*ncols, figsize_per_panel[1]*nrows))
    
    # Handle single subplot case
    if nrows == 1 and ncols == 1:
        axes = [axes]
    elif nrows == 1 or ncols == 1:
        axes = axes.flatten()
    else:
        axes = axes.flatten()
    
    # Plot each technology separately
    for i, tech in enumerate(technologies):
        ax = axes[i]
        
        # Subset data for this technology
        tech_mask = adata.obs[batch_key] == tech
        tech_coords = adata.obsm['X_umap'][tech_mask]
        tech_celltypes = adata.obs.loc[tech_mask, color_key]
        
        # Plot each cell type with consistent colors
        for cell_type in cell_types:
            cell_mask = tech_celltypes == cell_type
            if np.sum(cell_mask) > 0:  # Only plot if there are cells of this type
                ax.scatter(
                    tech_coords[cell_mask, 0], 
                    tech_coords[cell_mask, 1],
                    color=color_dict[cell_type],
                    s=1, alpha=0.7, label=cell_type
                )
        
        # Set consistent limits for all subplots
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        
        # Set labels and title
        ax.set_xlabel('UMAP1')
        ax.set_ylabel('UMAP2')
        ax.set_title(f'{batch_key.capitalize()}: {tech}')
        ax.set_aspect('equal')
    
    # Hide any unused subplots
    for i in range(n_techs, len(axes)):
        axes[i].set_visible(False)
    
    # Add a legend to the figure (outside the plots)
    handles, labels = [], []
    for cell_type in cell_types:
        handles.append(plt.Line2D([0], [0], marker='o', color=color_dict[cell_type], 
                                 label=cell_type, markersize=5, linestyle='None'))
        labels.append(cell_type)
    
    fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.15, 0.5))
    plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust layout to make room for legend
    plt.show()
    
    # Also create a combined plot with all technologies
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)
    
    # Plot colored by technology/batch
    sc.pl.umap(adata, color=batch_key, ax=ax1, frameon=False, show=False)
    ax1.set_xlim(x_min, x_max)
    ax1.set_ylim(y_min, y_max)
    ax1.set_title(f'Colored by {batch_key.capitalize()}')
    
    # Plot colored by celltype with consistent colors
    for cell_type in cell_types:
        cell_mask = adata.obs[color_key] == cell_type
        if np.sum(cell_mask) > 0:
            ax2.scatter(
                adata.obsm['X_umap'][cell_mask, 0],
                adata.obsm['X_umap'][cell_mask, 1],
                color=color_dict[cell_type],
                s=1, alpha=0.7, label=cell_type
            )
    
    ax2.set_xlim(x_min, x_max)
    ax2.set_ylim(y_min, y_max)
    ax2.set_title(f'Colored by {color_key.capitalize()}')
    
    # Add legend to the second plot
    handles, labels = [], []
    for cell_type in cell_types:
        handles.append(plt.Line2D([0], [0], marker='o', color=color_dict[cell_type], 
                                 label=cell_type, markersize=5, linestyle='None'))
        labels.append(cell_type)
    
    fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.15, 0.5))
    plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust layout to make room for legend
    plt.show()

In [ ]:
def compute_lisi(X, metadata, label_colname, perplexity=30):
    """
    Compute Local Inverse Simpson Index (LISI) for batch mixing evaluation.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        The embedded data matrix
    metadata : pandas.DataFrame
        Metadata containing batch/label information
    label_colname : str
        Column name in metadata containing the batch labels
    perplexity : int, default=30
        Perplexity parameter for Gaussian kernel (similar to t-SNE)
    
    Returns:
    --------
    lisi_scores : array-like
        LISI score for each cell
    """
    n_cells = X.shape[0]
    
    # Get batch labels
    batch_labels = metadata[label_colname].values
    unique_batches = np.unique(batch_labels)
    n_batches = len(unique_batches)
    
    # Create mapping from batch to index
    batch_to_idx = {batch: idx for idx, batch in enumerate(unique_batches)}
    batch_indices = np.array([batch_to_idx[batch] for batch in batch_labels])
    
    # Find k-nearest neighbors (k should be larger than perplexity)
    k = min(90, n_cells - 1)  # Use 90 neighbors or n_cells-1 if smaller
    print(f"Computing {k} nearest neighbors for {n_cells} cells...")
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(X)
    distances, indices = nbrs.kneighbors(X)
    
    lisi_scores = np.zeros(n_cells)
    
    # Add progress bar for LISI computation
    print(f"Computing LISI scores for {label_colname}...")
    for i in range(n_cells):
        # Get neighbors and distances for current cell
        neighbor_indices = indices[i, 1:]  # Exclude self (index 0)
        neighbor_distances = distances[i, 1:]
        
        # Compute Gaussian kernel weights with adaptive bandwidth
        # Find bandwidth that gives desired perplexity
        sigma = find_sigma(neighbor_distances, perplexity)
        weights = np.exp(-neighbor_distances**2 / (2 * sigma**2))
        weights = weights / np.sum(weights)  # Normalize
        
        # Get batch labels of neighbors
        neighbor_batches = batch_indices[neighbor_indices]
        
        # Compute probability of each batch in neighborhood
        batch_probs = np.zeros(n_batches)
        for j, batch_idx in enumerate(neighbor_batches):
            batch_probs[batch_idx] += weights[j]
        
        # Avoid division by zero
        batch_probs = batch_probs + 1e-12
        
        # Compute Simpson diversity (inverse Simpson index)
        simpson_index = np.sum(batch_probs**2)
        lisi_scores[i] = 1.0 / simpson_index
    
    return lisi_scores

def find_sigma(distances, target_perplexity, tol=1e-5, max_iter=50):
    """
    Find the Gaussian kernel bandwidth (sigma) that achieves target perplexity.
    Uses binary search similar to t-SNE implementation.
    """
    def perplexity_fn(sigma):
        if sigma <= 0:
            return 0
        weights = np.exp(-distances**2 / (2 * sigma**2))
        weights = weights / np.sum(weights)
        # Avoid log(0)
        weights = np.maximum(weights, 1e-12)
        H = -np.sum(weights * np.log2(weights))
        return 2**H
    
    # Binary search for sigma
    sigma_min, sigma_max = 1e-20, 1000.0
    
    for _ in range(max_iter):
        sigma = (sigma_min + sigma_max) / 2.0
        perp = perplexity_fn(sigma)
        
        if abs(perp - target_perplexity) < tol:
            break
            
        if perp > target_perplexity:
            sigma_max = sigma
        else:
            sigma_min = sigma
    
    return sigma

def ilisi_graph(adata, batch_key, type="embed", use_rep="X_pca", perplexity=30):
    """
    Compute integration Local Inverse Simpson Index (iLISI) for an AnnData object.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object
    batch_key : str
        Key in adata.obs containing batch information
    type : str, default="embed"
        Type of data to use ("embed" for embeddings)
    use_rep : str, default="X_pca"
        Key in adata.obsm for the embedding to use
    perplexity : int, default=30
        Perplexity parameter for neighborhood definition
    
    Returns:
    --------
    float
        Normalized mean iLISI score across all cells (0-1 range)
    """
    if type == "embed":
        print("Using embed")
        if use_rep not in adata.obsm:
            raise ValueError(f"Embedding {use_rep} not found in adata.obsm")
        X = adata.obsm[use_rep]
    else:
        X = adata.X
    
    if batch_key not in adata.obs:
        raise ValueError(f"Batch key {batch_key} not found in adata.obs")
    
    # Get number of unique batches for normalization
    n_batches = len(adata.obs[batch_key].unique())
    
    # Compute LISI scores
    print("Computing LISI")
    lisi_scores = compute_lisi(X, adata.obs, batch_key, perplexity)
    
    # Normalize by number of batches (perfect mixing = 1.0, no mixing = 1/n_batches)
    normalized_scores = (lisi_scores - 1) / (n_batches - 1)
    
    # Return mean normalized iLISI score
    return np.mean(normalized_scores)

def clisi_graph(adata, label_key, type="embed", use_rep="X_pca", perplexity=30):
    """
    Compute cell-type Local Inverse Simpson Index (cLISI) for an AnnData object.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object
    label_key : str
        Key in adata.obs containing cell type information
    type : str, default="embed"
        Type of data to use ("embed" for embeddings)
    use_rep : str, default="X_pca"
        Key in adata.obsm for the embedding to use
    perplexity : int, default=30
        Perplexity parameter for neighborhood definition
    
    Returns:
    --------
    float
        Normalized mean cLISI score across all cells (0-1 range)
    """
    if type == "embed":
        print("Using embed")
        if use_rep not in adata.obsm:
            raise ValueError(f"Embedding {use_rep} not found in adata.obsm")
        X = adata.obsm[use_rep]
    else:
        X = adata.X
    
    if label_key not in adata.obs:
        raise ValueError(f"Label key {label_key} not found in adata.obs")
    
    # Get number of unique cell types for normalization
    n_celltypes = len(adata.obs[label_key].unique())
    
    print("Computing LISI")
    # Compute LISI scores
    lisi_scores = compute_lisi(X, adata.obs, label_key, perplexity)
    
    # Normalize by number of cell types (perfect mixing = 1.0, no mixing = 1/n_celltypes)
    normalized_scores = (lisi_scores - 1) / (n_celltypes - 1)
    
    # Return mean normalized cLISI score
    return np.mean(normalized_scores)

In [ ]:
# set the torch random seed
set_seed(42)

In [ ]:
adata = sc.read_h5ad("/workspaces/data/human_pancreas_norm_complexBatch.h5ad")

adata.raw = adata
adata.layers["counts"] = adata.X.copy()
sc.pp.filter_cells(adata, min_genes=300)
sc.pp.filter_genes(adata, min_cells=5)
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key='tech')
adata = adata[:, adata.var['highly_variable']]
multi_resolution_cluster(adata, resolution1 = 1, method = 'Leiden')

In [ ]:
plot_umap_by_technology(adata, batch_key='tech', color_key='celltype')

In [ ]:
model = train_integration_model(adata, batch_key = 'tech', z_dim=256, d_coef = 0.01, epochs=100, critic=True, disc_iter=10)
obtain_embeddings(adata, model.to("cuda:0"))
sc.pp.neighbors(adata, use_rep="X_scCRAFT")
plot_umap_by_technology(adata, batch_key='tech', color_key='celltype')

print(scib.me.silhouette(adata, label_key="celltype", embed="X_scCRAFT", scale=True))
print(scib.me.silhouette_batch(adata, batch_key="tech", label_key="celltype", embed="X_scCRAFT", scale=True))

ilisi_score = ilisi_graph(adata, batch_key="tech", type="embed", use_rep="X_scCRAFT")
print(f"iLISI score (1 is best): {ilisi_score:.4f}")

clisi_score = clisi_graph(adata, label_key="celltype", type="embed", use_rep="X_scCRAFT")
print(f"cLISI score (0 is best): {clisi_score:.4f}")

In [ ]:
model = train_integration_model(adata, batch_key = 'tech', z_dim=256, d_coef = 0.1, epochs=100, critic=False, disc_iter=10)
obtain_embeddings(adata, model.to("cuda:0"))
sc.pp.neighbors(adata, use_rep="X_scCRAFT")
plot_umap_by_technology(adata, batch_key='tech', color_key='celltype')

print(scib.me.silhouette(adata, label_key="celltype", embed="X_scCRAFT", scale=True))
print(scib.me.silhouette_batch(adata, batch_key="tech", label_key="celltype", embed="X_scCRAFT", scale=True))

ilisi_score = ilisi_graph(adata, batch_key="tech", type="embed", use_rep="X_scCRAFT")
print(f"iLISI score (1 is best): {ilisi_score:.4f}")

clisi_score = clisi_graph(adata, label_key="celltype", type="embed", use_rep="X_scCRAFT")
print(f"cLISI score (0 is best): {clisi_score:.4f}")

In [ ]:
from sklearn.model_selection import KFold
from scipy import stats
import numpy as np
import pandas as pd
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

def nested_cv_hyperparameter_tuning(adata, batch_key='tech', celltype_key='celltype', 
                                   d_coef_range=[0.01, 0.05, 0.1, 0.2, 0.5], 
                                   n_outer_folds=5, n_inner_folds=3, 
                                   z_dim=256, epochs=50, disc_iter=10, 
                                   random_state=42):
    """
    Perform nested cross-validation for hyperparameter tuning of d_coef with and without critic.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object
    batch_key : str
        Key for batch information
    celltype_key : str  
        Key for cell type information
    d_coef_range : list
        Range of d_coef values to test
    n_outer_folds : int
        Number of outer CV folds
    n_inner_folds : int
        Number of inner CV folds for hyperparameter selection
    z_dim : int
        Latent dimension
    epochs : int
        Training epochs
    disc_iter : int
        Discriminator iterations
    random_state : int
        Random seed
    
    Returns:
    --------
    results_df : DataFrame
        Results with outer fold statistics
    """
    
    set_seed(random_state)
    
    # Prepare data indices for stratified splitting by cell type
    cell_indices = np.arange(adata.n_obs)
    celltype_labels = adata.obs[celltype_key].values
    
    # Create stratified outer folds
    outer_kf = KFold(n_splits=n_outer_folds, shuffle=True, random_state=random_state)
    
    # Store results for each outer fold
    outer_fold_results = {
        'critic': {'ilisi': [], 'clisi': [], 'best_d_coef': []},
        'no_critic': {'ilisi': [], 'clisi': [], 'best_d_coef': []}
    }
    
    print(f"Starting nested {n_outer_folds}-fold CV with {len(d_coef_range)} d_coef values...")
    print(f"d_coef range: {d_coef_range}")
    
    for outer_fold, (train_idx, test_idx) in enumerate(outer_kf.split(cell_indices)):
        print(f"\n=== OUTER FOLD {outer_fold + 1}/{n_outer_folds} ===")
        
        # Split data for outer fold
        adata_train = adata[train_idx].copy()
        adata_test = adata[test_idx].copy()
        
        print(f"Train set: {adata_train.n_obs} cells, Test set: {adata_test.n_obs} cells")
        
        # Inner CV for hyperparameter selection (both critic and no_critic)
        for use_critic in [True, False]:
            critic_label = 'critic' if use_critic else 'no_critic'
            print(f"\n--- {critic_label.upper()} ---")
            
            # Inner cross-validation for hyperparameter selection
            inner_kf = KFold(n_splits=n_inner_folds, shuffle=True, random_state=random_state)
            inner_scores = defaultdict(list)
            
            print("Inner CV for hyperparameter selection...")
            for d_coef in d_coef_range:
                print(f"  Testing d_coef={d_coef}")
                
                # Inner fold validation scores
                inner_ilisi_scores = []
                inner_clisi_scores = []
                
                for inner_fold, (inner_train_idx, inner_val_idx) in enumerate(inner_kf.split(train_idx)):
                    # Get actual indices
                    actual_train_idx = train_idx[inner_train_idx]
                    actual_val_idx = train_idx[inner_val_idx]
                    
                    adata_inner_train = adata[actual_train_idx].copy()
                    adata_inner_val = adata[actual_val_idx].copy()
                    
                    try:
                        # Train model on inner training set
                        model = train_integration_model(
                            adata_inner_train, 
                            batch_key=batch_key,
                            z_dim=z_dim, 
                            d_coef=d_coef, 
                            epochs=epochs, 
                            critic=use_critic, 
                            disc_iter=disc_iter
                        )
                        
                        # Get embeddings for validation set
                        device = "cuda:0" if torch.cuda.is_available() else "cpu"
                        obtain_embeddings(adata_inner_val, model.to(device))
                        
                        # Compute validation scores
                        ilisi_val = ilisi_graph(adata_inner_val, batch_key=batch_key, 
                                              type="embed", use_rep="X_scCRAFT")
                        clisi_val = clisi_graph(adata_inner_val, label_key=celltype_key, 
                                              type="embed", use_rep="X_scCRAFT")
                        
                        inner_ilisi_scores.append(ilisi_val)
                        inner_clisi_scores.append(clisi_val)
                        
                    except Exception as e:
                        print(f"    Error in inner fold {inner_fold}: {e}")
                        inner_ilisi_scores.append(0.0)
                        inner_clisi_scores.append(1.0)
                
                # Average scores across inner folds
                avg_ilisi = np.mean(inner_ilisi_scores)
                avg_clisi = np.mean(inner_clisi_scores)
                
                # Composite score (higher iLISI is better, lower cLISI is better)
                composite_score = avg_ilisi - avg_clisi
                
                inner_scores[d_coef] = {
                    'ilisi': avg_ilisi,
                    'clisi': avg_clisi, 
                    'composite': composite_score
                }
                
                print(f"    d_coef={d_coef}: iLISI={avg_ilisi:.4f}, cLISI={avg_clisi:.4f}, composite={composite_score:.4f}")
            
            # Select best hyperparameter based on composite score
            best_d_coef = max(inner_scores.keys(), key=lambda k: inner_scores[k]['composite'])
            print(f"  Best d_coef for {critic_label}: {best_d_coef}")
            
            # Train final model on full training set with best hyperparameter
            print(f"  Training final model with d_coef={best_d_coef}")
            try:
                final_model = train_integration_model(
                    adata_train, 
                    batch_key=batch_key,
                    z_dim=z_dim, 
                    d_coef=best_d_coef, 
                    epochs=epochs, 
                    critic=use_critic, 
                    disc_iter=disc_iter
                )
                
                # Evaluate on test set
                device = "cuda:0" if torch.cuda.is_available() else "cpu"
                obtain_embeddings(adata_test, final_model.to(device))
                
                # Compute test scores
                test_ilisi = ilisi_graph(adata_test, batch_key=batch_key, 
                                       type="embed", use_rep="X_scCRAFT")
                test_clisi = clisi_graph(adata_test, label_key=celltype_key, 
                                       type="embed", use_rep="X_scCRAFT")
                
                print(f"  Test scores: iLISI={test_ilisi:.4f}, cLISI={test_clisi:.4f}")
                
                # Store results
                outer_fold_results[critic_label]['ilisi'].append(test_ilisi)
                outer_fold_results[critic_label]['clisi'].append(test_clisi)
                outer_fold_results[critic_label]['best_d_coef'].append(best_d_coef)
                
            except Exception as e:
                print(f"  Error in final training: {e}")
                outer_fold_results[critic_label]['ilisi'].append(0.0)
                outer_fold_results[critic_label]['clisi'].append(1.0)
                outer_fold_results[critic_label]['best_d_coef'].append(best_d_coef)
    
    # Create results DataFrame
    results_data = []
    for fold in range(n_outer_folds):
        for critic_type in ['critic', 'no_critic']:
            results_data.append({
                'fold': fold + 1,
                'method': critic_type,
                'ilisi': outer_fold_results[critic_type]['ilisi'][fold],
                'clisi': outer_fold_results[critic_type]['clisi'][fold],
                'best_d_coef': outer_fold_results[critic_type]['best_d_coef'][fold]
            })
    
    results_df = pd.DataFrame(results_data)
    
    return results_df, outer_fold_results

def analyze_cv_results(results_df, outer_fold_results):
    """
    Analyze and compare CV results with statistical tests.
    """
    
    print("\n" + "="*60)
    print("NESTED CROSS-VALIDATION RESULTS ANALYSIS")
    print("="*60)
    
    # Summary statistics
    summary_stats = results_df.groupby('method').agg({
        'ilisi': ['mean', 'std', 'min', 'max'],
        'clisi': ['mean', 'std', 'min', 'max'],
        'best_d_coef': lambda x: list(x)
    }).round(4)
    
    print("\nSUMMARY STATISTICS:")
    print(summary_stats)
    
    # Statistical tests
    critic_ilisi = outer_fold_results['critic']['ilisi']
    no_critic_ilisi = outer_fold_results['no_critic']['ilisi']
    critic_clisi = outer_fold_results['critic']['clisi']
    no_critic_clisi = outer_fold_results['no_critic']['clisi']
    
    print(f"\nSTATISTICAL TESTS (n={len(critic_ilisi)} folds each):")
    print("-" * 50)
    
    # Paired t-test for iLISI (higher is better)
    ilisi_tstat, ilisi_pval = stats.ttest_rel(critic_ilisi, no_critic_ilisi)
    print(f"iLISI Paired t-test:")
    print(f"  Critic mean ± std: {np.mean(critic_ilisi):.4f} ± {np.std(critic_ilisi):.4f}")
    print(f"  No-critic mean ± std: {np.mean(no_critic_ilisi):.4f} ± {np.std(no_critic_ilisi):.4f}")
    print(f"  t-statistic: {ilisi_tstat:.4f}")
    print(f"  p-value: {ilisi_pval:.6f}")
    print(f"  Significant: {'Yes' if ilisi_pval < 0.05 else 'No'}")
    if ilisi_pval < 0.05:
        better_ilisi = "Critic" if np.mean(critic_ilisi) > np.mean(no_critic_ilisi) else "No-critic"
        print(f"  {better_ilisi} performs significantly better for iLISI")
    
    # Paired t-test for cLISI (lower is better)
    clisi_tstat, clisi_pval = stats.ttest_rel(critic_clisi, no_critic_clisi)
    print(f"\ncLISI Paired t-test:")
    print(f"  Critic mean ± std: {np.mean(critic_clisi):.4f} ± {np.std(critic_clisi):.4f}")
    print(f"  No-critic mean ± std: {np.mean(no_critic_clisi):.4f} ± {np.std(no_critic_clisi):.4f}")
    print(f"  t-statistic: {clisi_tstat:.4f}")
    print(f"  p-value: {clisi_pval:.6f}")
    print(f"  Significant: {'Yes' if clisi_pval < 0.05 else 'No'}")
    if clisi_pval < 0.05:
        better_clisi = "Critic" if np.mean(critic_clisi) < np.mean(no_critic_clisi) else "No-critic"
        print(f"  {better_clisi} performs significantly better for cLISI")
    
    # Effect sizes (Cohen's d)
    def cohens_d(x, y):
        nx, ny = len(x), len(y)
        dof = nx + ny - 2
        pooled_std = np.sqrt(((nx-1)*np.var(x, ddof=1) + (ny-1)*np.var(y, ddof=1)) / dof)
        return (np.mean(x) - np.mean(y)) / pooled_std
    
    ilisi_cohens_d = cohens_d(critic_ilisi, no_critic_ilisi)
    clisi_cohens_d = cohens_d(no_critic_clisi, critic_clisi)  # Flipped for "lower is better"
    
    print(f"\nEFFECT SIZES (Cohen's d):")
    print(f"  iLISI Cohen's d: {ilisi_cohens_d:.4f}")
    print(f"  cLISI Cohen's d: {clisi_cohens_d:.4f}")
    
    def interpret_cohens_d(d):
        if abs(d) < 0.2:
            return "negligible"
        elif abs(d) < 0.5:
            return "small"
        elif abs(d) < 0.8:
            return "medium"
        else:
            return "large"
    
    print(f"  iLISI effect size: {interpret_cohens_d(ilisi_cohens_d)}")
    print(f"  cLISI effect size: {interpret_cohens_d(clisi_cohens_d)}")
    
    # Wilcoxon signed-rank test (non-parametric alternative)
    ilisi_wilc_stat, ilisi_wilc_pval = stats.wilcoxon(critic_ilisi, no_critic_ilisi)
    clisi_wilc_stat, clisi_wilc_pval = stats.wilcoxon(critic_clisi, no_critic_clisi)
    
    print(f"\nWILCOXON SIGNED-RANK TESTS (non-parametric):")
    print(f"  iLISI p-value: {ilisi_wilc_pval:.6f}")
    print(f"  cLISI p-value: {clisi_wilc_pval:.6f}")
    
    # Hyperparameter frequency analysis
    print(f"\nHYPERPARAMETER SELECTION FREQUENCY:")
    critic_d_coef_freq = pd.Series(outer_fold_results['critic']['best_d_coef']).value_counts()
    no_critic_d_coef_freq = pd.Series(outer_fold_results['no_critic']['best_d_coef']).value_counts()
    
    print("Critic:")
    for d_coef, freq in critic_d_coef_freq.items():
        print(f"  d_coef={d_coef}: {freq}/{len(critic_ilisi)} folds")
    
    print("No-critic:")
    for d_coef, freq in no_critic_d_coef_freq.items():
        print(f"  d_coef={d_coef}: {freq}/{len(no_critic_ilisi)} folds")
    
    # Overall conclusion
    print(f"\nOVERALL CONCLUSION:")
    print("-" * 20)
    
    # Count significant improvements
    significant_improvements = 0
    if ilisi_pval < 0.05 and np.mean(critic_ilisi) > np.mean(no_critic_ilisi):
        significant_improvements += 1
        print("✓ Critic significantly better for batch mixing (iLISI)")
    elif ilisi_pval < 0.05:
        print("✗ No-critic significantly better for batch mixing (iLISI)")
    else:
        print("≈ No significant difference in batch mixing (iLISI)")
    
    if clisi_pval < 0.05 and np.mean(critic_clisi) < np.mean(no_critic_clisi):
        significant_improvements += 1
        print("✓ Critic significantly better for cell type preservation (cLISI)")
    elif clisi_pval < 0.05:
        print("✗ No-critic significantly better for cell type preservation (cLISI)")
    else:
        print("≈ No significant difference in cell type preservation (cLISI)")
    
    if significant_improvements == 2:
        print("\n🎉 CRITIC APPROACH SHOWS SIGNIFICANT IMPROVEMENT IN BOTH METRICS!")
    elif significant_improvements == 1:
        print("\n⚖️  CRITIC APPROACH SHOWS MIXED RESULTS (better in one metric)")
    else:
        print("\n❌ CRITIC APPROACH DOES NOT SHOW SIGNIFICANT IMPROVEMENT")
    
    return summary_stats

print("Starting nested cross-validation...")
results_df, outer_fold_results = nested_cv_hyperparameter_tuning(
    adata,
    batch_key='tech',
    celltype_key='celltype', 
    d_coef_range=[0.01, 0.05, 0.1, 0.2],  # Reduced range for faster execution
    n_outer_folds=5,
    n_inner_folds=3,
    z_dim=128,      # Reduced for faster training
    epochs=30,      # Reduced for faster training  
    disc_iter=5,    # Reduced for faster training
    random_state=42
)

# Analyze results
summary_stats = analyze_cv_results(results_df, outer_fold_results)

# Save results
results_df.to_csv('nested_cv_results.csv', index=False)
print(f"\nResults saved to 'nested_cv_results.csv'")